# TEXT MINING for PRACTICE: Document Clustering
- 출처 : https://ratsgo.github.io/from%20frequency%20to%20semantics/2017/07/09/lda/
---


In [61]:
from konlpy.tag import Okt
okt = Okt()
# tokenizer : 문장에서 색인어 추출을 위해 명사,동사,알파벳,숫자 정도의 단어만 뽑아서 normalization, stemming 처리하도록 함
def tokenizer(raw, pos=["Noun"], stopword=[]):
    return [
        word for word, tag in okt.pos(
            raw, 
            norm=True,   # normalize 그랰ㅋㅋ -> 그래ㅋㅋ
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword
        ]


In [62]:
import pandas as pd
data = pd.read_csv("data/article_sample.txt", sep="\t", names=["topic","source","date","title","content"])
data.head()

,topic,source,date,title,content
0,비트코인,ZDNet Korea,2019-04-14 09:58:00,"비트코인 가능성 알려면, 결함 전제로 규제 정비必",금융硏 비트코인 쇠락과 내재적 결함 보고서(지디넷코리아=손예술 기자)대표적인 암호화...
1,비트코인,한국경제,2019-03-09 07:02:00,"워렌 버핏 ""비트코인은 망상"" vs 하버드대 교수 ""망상 아니다""",비트코인 놓고 상반된 평가 내놔[ 김산하 기자 ] 니얼 퍼거슨 하버드대 교수(왼쪽...
2,비트코인,한국경제,2019-04-07 09:02:00,"가상화폐, 94년 인터넷 수준…비트코인캐시가 선두주자 될 것",[코인터뷰] 스테판 러스트 비트코인닷컴 비즈니스총괄[ 김산하 기자 ] 한경닷컴과 ...
3,비트코인,ZDNet Korea,2019-01-03 12:39:00,[비트코인 10주년 ]제네시스 블록 탄생에서 크립토윈터까지,10년 역사 10대 뉴스로 정리(지디넷코리아=임유경 기자)2008년 9월 리먼 브라...
4,비트코인,아시아경제,2019-04-22 11:20:00,"[아시아미래기업포럼]이준행 고팍스 대표 ""비트코인이 금융시스템 혁신""",세상 긍정적 변화시킬 기술아시아경제주최로 22일 서울 중구 대한상의에서 열린 '20...


In [63]:
documents = []
for content in data['content'].values:
    documents.append(tokenizer(content))

In [64]:
import random

In [70]:
from collections import Counter
# Topic 수
K=3

# 각 토픽이 각 문서에 할당되는 횟수
# Counter로 구성된 리스트
# 각 Counter는 각 문서를 의미
document_topic_counts = [Counter() for _ in documents]

# 각 단어가 각 토픽에 할당되는 횟수
# Counter로 구성된 리스트
# 각 Counter는 각 토픽을 의미
topic_word_counts = [Counter() for _ in range(K)]

# 각 토픽에 할당되는 총 단어수
# 숫자로 구성된 리스트
# 각각의 숫자는 각 토픽을 의미함
topic_counts = [0 for _ in range(K)]

# 각 문서에 포함되는 총 단어수
# 숫자로 구성된 리스트
# 각각의 숫자는 각 문서를 의미함
document_lengths = list(map(len, documents))

# 단어 종류의 수
distinct_words = set(word for document in documents for word in document)
V = len(distinct_words)

# 총 문서의 수
D = len(documents)

In [71]:
def p_topic_given_document(topic, d, alpha=0.1):
    # 문서 d의 모든 단어 가운데 topic에 속하는
    # 단어의 비율 (alpha를 더해 smoothing)
    return ((document_topic_counts[d][topic] + alpha) /
            (document_lengths[d] + K * alpha))

def p_word_given_topic(word, topic, beta=0.1):
    # topic에 속한 단어 가운데 word의 비율
    # (beta를 더해 smoothing)
    return ((topic_word_counts[topic][word] + beta) /
            (topic_counts[topic] + V * beta))

def topic_weight(d, word, k):
    # 문서와 문서의 단어가 주어지면
    # k번째 토픽의 weight를 반환
    return p_word_given_topic(word, k) * p_topic_given_document(k, d)

In [72]:
def choose_new_topic(d, word):
    return sample_from([topic_weight(d, word, k) for k in range(K)])

import random
def sample_from(weights):
    # i를 weights[i] / sum(weights)
    # 확률로 반환
    total = sum(weights)
    # 0과 total 사이를 균일하게 선택
    rnd = total * random.random()
    # 아래 식을 만족하는 가장 작은 i를 반환
    # weights[0] + ... + weights[i] >= rnd
    for i, w in enumerate(weights):
        rnd -= w
        if rnd <= 0:
            return i

In [73]:
random.seed(0)



# 각 단어를 임의의 토픽에 랜덤 배정
document_topics = [[random.randrange(K) for word in document]
                    for document in documents]

# 위와 같이 랜덤 초기화한 상태에서 
# AB를 구하는 데 필요한 숫자를 세어봄
for d in range(D):
    for word, topic in zip(documents[d], document_topics[d]):
        document_topic_counts[d][topic] += 1
        topic_word_counts[topic][word] += 1
        topic_counts[topic] += 1

In [74]:
for iter in range(1000):
    for d in range(D):
        for i, (word, topic) in enumerate(zip(documents[d],
                                              document_topics[d])):
            # 깁스 샘플링 수행을 위해
            # 샘플링 대상 word와 topic을 제외하고 세어봄
            document_topic_counts[d][topic] -= 1
            topic_word_counts[topic][word] -= 1
            topic_counts[topic] -= 1
            document_lengths[d] -= 1

            # 깁스 샘플링 대상 word와 topic을 제외한 
            # 말뭉치 모든 word의 topic 정보를 토대로
            # 샘플링 대상 word의 새로운 topic을 선택
            new_topic = choose_new_topic(d, word)
            document_topics[d][i] = new_topic

            # 샘플링 대상 word의 새로운 topic을 반영해 
            # 말뭉치 정보 업데이트
            document_topic_counts[d][new_topic] += 1
            topic_word_counts[new_topic][word] += 1
            topic_counts[new_topic] += 1
            document_lengths[d] += 1

In [79]:
topic_word_counts[0].most_common(n=10)

[('부동산', 518),
 ('금융', 410),
 ('신탁', 139),
 ('그림자', 136),
 ('시장', 91),
 ('관련', 90),
 ('규모', 88),
 ('투자', 80),
 ('국내', 68),
 ('리스크', 60)]

In [80]:
topic_word_counts[1].most_common(n=10)

[('비트코인', 541),
 ('화폐', 206),
 ('암호', 181),
 ('금융', 123),
 ('자산', 108),
 ('블록', 101),
 ('체인', 100),
 ('거래', 95),
 ('서비스', 72),
 ('디지털', 71)]

In [81]:
topic_word_counts[2].most_common(n=10)

[('금리', 777),
 ('대출', 475),
 ('은행', 230),
 ('코픽스', 207),
 ('금융', 206),
 ('기준', 195),
 ('당국', 122),
 ('잔액', 93),
 ('평균', 93),
 ('산정', 89)]